**Customer Lifetime Value analysis**

resq-data-assignment-2024
Topias Pesonen
topias.pesonen@gmail.com

Problem: How much to spend on acquiring new customers?

*What is the expected lifetime on the platform, and how much do they bring in on average*

Factors to consider:
- purchase frequency (simplify to number of purchases *ever*, not for example how often in a year?)
- average order value
- average customer lifespan (for this we might use some survival analysis method, but that's for another day)


Questions to consider:
- should we remove refunds from sales?
- we should use net profit instead of revenue

In [122]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

conn = sqlite3.connect('mock_resq.db')

df = pd.read_sql_query("SELECT * FROM presentation_table", conn)
conn.close()

# Change dates to datetime
df['cohort'] = pd.to_datetime(df['cohort'])
df['createdAt'] = pd.to_datetime(df['createdAt'])

print(df.head())
print()
print(df.info())

                id               userID     cohort  M1_retention  \
0   15079796150070  1651090691146392618 2022-10-01             1   
1   81169833397043  7338126717742165249 2022-11-01             0   
2  126451530207724  4654198358962600528 2023-07-01             1   
3  131192980331210  2157947595893547130 2022-12-01             0   
4  171979511824449  7588211493098786060 2022-10-01             1   

            createdAt   sales              partner segment  
0 2023-08-01 08:01:14   650.0  3158809927039353473    meal  
1 2023-10-13 07:23:57   150.0  3186777369203625736    meal  
2 2023-08-21 10:38:54  1000.0  1544951218129901618    meal  
3 2023-08-15 12:33:50   800.0  7014807967414306862    meal  
4 2023-06-07 05:44:16   200.0  1688430276435400909    meal  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299971 entries, 0 to 299970
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   id     

In [123]:
# Let's start with average order value, since it's the simplest

aov = df['sales'].mean() / 100
print(f'Average order value: {aov:.2f} euros')

Average order value: 7.70 euros


In [124]:
# Purchase frequency
# We are interested in how many puchases an average customer makes 

per_customer = df.groupby('userID').size()
avg_purchases = per_customer.mean()
print(f'The average customer makes {avg_purchases:.3f} purchases in their lifetime')
# note: this also works: len(df) / len(df.groupby('userID').size())

counts_df = df['userID'].value_counts().reset_index(name='counts')

# normalized df shows the % of each order count
norm_counts_df = counts_df['counts'].value_counts(normalize=True)
print()
print('n orders and their percentage')
print(norm_counts_df)
print()

prop_only_one_order = counts_df['counts'].value_counts(normalize=True)[1]
print(f'{100 * prop_only_one_order:.2f}% of customers have made only one order')
print(f'{100*counts_df['counts'].value_counts(normalize=True)[1:5].sum():.2f}% of customers have made between 2 and 5 orders')
print(f'{100*counts_df['counts'].value_counts(normalize=True)[5:].sum():.2f}% of customers have made more than 5 orders')

# recurring customers df of those who return for a second order
rc_df = df[df['userID'].isin(counts_df[counts_df['counts'] > 1]['userID'])]
#rc_df 

The average customer makes 2.435 purchases in their lifetime

n orders and their percentage
counts
1      0.539421
2      0.197300
3      0.094961
4      0.053633
5      0.032189
         ...   
72     0.000008
76     0.000008
95     0.000008
57     0.000008
266    0.000008
Name: proportion, Length: 63, dtype: float64

53.94% of customers have made only one order
37.81% of customers have made between 2 and 5 orders
8.25% of customers have made more than 5 orders


In [125]:
# average customer lifespan 
# we can use the recurring customers and calculate:
# last order date - first order date

user_lifespans = rc_df.groupby('userID').agg({
    'createdAt': [
        ('first_purchase', 'min'),
        ('last_purchase', 'max'),
        ('purchase_count', 'count')
    ]
})
user_lifespans.columns = ['first_purchase', 'last_purchase', 'purchase_count']
user_lifespans['lifespan'] = (user_lifespans['last_purchase'] - user_lifespans['first_purchase']).dt.days
avg_lifespan_days = user_lifespans['lifespan'].mean()

print(f"Average lifespan for recurring customers: {avg_lifespan_days:.2f} days")

Average lifespan for recurring customers: 167.12 days


In [126]:
# Customer Lifetime Value
# CLV = average order value * average purchases

clv = aov * avg_purchases
print(f'Average Customer Lifetime Value: {clv:.2f} euros')
print(f"Average Revenue per day: {clv / avg_lifespan_days:.2f} euros")
print(f"Average time between purchases: {avg_lifespan_days / avg_purchases:.2f} days")


Average Customer Lifetime Value: 18.76 euros
Average revenue per day: 0.11 euros
Average time between purchases: 68.64 days
